# Auto-Roleplay

In this simple notebook, we create a few agents that we will use to simulate a roleplay between different characters and a narrator!

The concept is simple: Having a Game Master Agent that will decide whose turn it is, here for simplicity will be between a Narrator Agent or a Character Agent.

Then the Narrator or Character will answer, and loop this cycle to infinity!

## Agents
You can create an Agent in https://console.mistral.ai/build/agents/new, for this notebook we will use `mistral-large-2407` as the model powering our agents!

### Game Master
The Game Master will be the one in charge of handling all the main decisions and creation of the characters. We want it to be deterministic and accurate mostly when deciding who should be the next to answer while respecting accurately the format required. Hence we will set a low `temperature of 0`.

Here are the instructions provided to the agent:
```
You are a Game Master in charge of guiding a roleplaying story.
You decide if the next step should be:
- Narrator
- Character

If you decide it's the Narrator's turn, return "Narrator" with the following JSON:
{{
  "choice":"Narrator",
}}
If you decide it's a Character's or Creature's turn, return "Character". Followed by a description and name of the character/creature. You can either use a previous character/creature or introduce a new one. With the following JSON format:
{{
  "choice":"Character",
  "character_description":
  {{
    "name":"character's/creature's name",
    "other":"information about the character/creature, personality, appearance, and other information"
  }},
}}
```

### Narrator
The Narrator, on the other hand, will be the one narrating our story and will have a more surface-level narration and guide the story. We want the story to be very pleasant and original, to add creativity randomness plays well here, hence we will set a high `temperature of 0.8`.

Here are the instructions provided to the agent:
```
Your role is to narrate the roleplaying story. You only narrate, the narration must be between in **bold** with markdown. The narration should be short and fit in a single paragraph.
```

### Character
Finally, our Character agent needs to be able to roleplay as any character given a description, while also following a specific desired format. While this might require a decent determinism, we also do not want it to be too monotone and repetitive, so a `temperature of 0.5` sounds good!

Here are the instructions provided to the agent:
```
Your role is to roleplay as a character by respecting the JSON with the description and name.
Answer always with the following format:
**Actions and thoughts in bold.**
- "Dialogs followed by a hyphen and quotes"
```

## Coding our workflow
We can now make our Agents collaborate to harmoniously have a roleplaying simulation between agents.

### Install dependencies
First we will install the python SDK and set our API key!

In [ ]:
!pip install mistralai==1.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.8/214.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.2
    Uninstalling python-dateutil-2.8.2:
      Successfully uninstalled python-dateutil-2.8.2


In [ ]:
from mistralai import Mistral
client = Mistral(api_key="api_key")

### Agents IDs
Next, we will retrieve the Agents IDs from the UI where we created the agents.

In [ ]:
game_master_id = "ag:8e2706f0:20240806:game-master:6d101f5a"
narrator_id = "ag:8e2706f0:20240806:narrator:388b4fa8"
character_id = "ag:8e2706f0:20240806:character:e5716606"

### Run the simulation

In [ ]:
import json

plot = "A fantasy story with different creatures and magic." # the main plot of the story
messages = [{"role": "user", "content": plot}]

for i in range(10): # simulate 10 messages

  resp = client.agents.complete(agent_id=game_master_id, messages=messages, response_format={"type":"json_object"}) # the Game Master decides what to do
  choice = json.loads(resp["choices"][0]["message"]["content"])

  match choice["choice"]:
    case "Narrator":
      resp = client.agents.complete(agent_id=narrator_id, messages=messages)
      messages.append({"role": "assistant", "content": resp["choices"][0]["message"]["content"]})

      print("Narrator:", resp["choices"][0]["message"]["content"])

    case "Character":
      messages[-1]["content"] += "\n\n" + json.dumps(choice["character_description"]) # adds the character description made by the Game Master for the next interaction

      resp = client.agents.complete(agent_id=character_id, messages=messages)
      messages.append({"role": "assistant", "content": resp["choices"][0]["message"]["content"]})

      print(choice["character_description"]["name"]+":",resp["choices"][0]["message"]["content"])

  messages = [{"role":"user" if m["role"] == "assistant" else "assistant", "content": m["content"]} for m in messages]